In [4]:
import os
import pandas as pd
from config import results_dir
print(results_dir)

z:/cache/MR\Results


In [8]:
res = pd.read_csv(os.path.join(results_dir, 'prot_vs_prolapse.csv'))

res.head()

,id.exposure,id.outcome,outcome,exposure,method,nsnp,b,se,pval
0,prot-a-1,finn-a-N14_FEMGENPROL,Female genital prolapse || id:finn-a-N14_FEMGE...,APOBEC1 complementation factor || id:prot-a-1,Wald ratio,1,0.128470,0.084355,0.127769
1,prot-a-1,ukb-a-577,Diagnoses - main ICD10: N81 Female genital pro...,APOBEC1 complementation factor || id:prot-a-1,Wald ratio,1,-0.001216,0.000984,0.216618
2,prot-a-1,ukb-a-92,Non-cancer illness code self-reported: vagina...,APOBEC1 complementation factor || id:prot-a-1,Wald ratio,1,-0.000310,0.000617,0.615021
3,prot-a-1,ukb-b-15904,"Non-cancer illness code, self-reported: prolap...",APOBEC1 complementation factor || id:prot-a-1,Wald ratio,1,-0.001758,0.000912,0.053965
4,prot-a-1,ukb-b-175,Operation code: vaginal prolapse/colposuspensi...,APOBEC1 complementation factor || id:prot-a-1,Wald ratio,1,0.000218,0.000678,0.747302


In [17]:
filt_res = res.query("nsnp > 1 and pval < 0.05")
counts = filt_res.groupby(["id.exposure", "id.outcome"])['b'].count()
filt_index = counts[counts > 1].index
filt_res = filt_res.set_index(["id.exposure", "id.outcome"]).loc[filt_index]
filt_res

outcome  \
id.exposure id.outcome                                                       
prot-a-1005 ukb-a-577    Diagnoses - main ICD10: N81 Female genital pro...   
            ukb-a-577    Diagnoses - main ICD10: N81 Female genital pro...   
prot-a-1045 ukb-a-92     Non-cancer illness code  self-reported: vagina...   
            ukb-a-92     Non-cancer illness code  self-reported: vagina...   
prot-a-1072 ukb-b-175    Operation code: vaginal prolapse/colposuspensi...   
...                                                                    ...   
prot-a-794  ukb-a-92     Non-cancer illness code  self-reported: vagina...   
            ukb-b-11282  Non-cancer illness code, self-reported: vagina...   
            ukb-b-11282  Non-cancer illness code, self-reported: vagina...   
prot-a-857  ukb-b-15904  Non-cancer illness code, self-reported: prolap...   
            ukb-b-15904  Non-cancer illness code, self-reported: prolap...   

                                                                  exposure  \
id.exposure id.outcome                                                       
prot-a-1005 ukb-a-577              Coagulation factor Xa || id:prot-a-1005   
            ukb-a-577              Coagulation factor Xa || id:prot-a-1005   
prot-a-1045 ukb-a-92                 Pseudokinase FAM20A || id:prot-a-1045   
            ukb-a-92                 Pseudokinase FAM20A || id:prot-a-1045   
prot-a-1072 ukb-b-175    Low affinity immunoglobulin epsilon Fc recepto...   
...                                                                    ...   
prot-a-794  ukb-a-92                    Beta-defensin 119 || id:prot-a-794   
            ukb-b-11282                 Beta-defensin 119 || id:prot-a-794   
            ukb-b-11282                 Beta-defensin 119 || id:prot-a-794   
prot-a-857  ukb-b-15904                     Dipeptidase 2 || id:prot-a-857   
            ukb-b-15904                     Dipeptidase 2 || id:prot-a-857   

                                            method  nsnp         b        se  \
id.exposure id.outcome                                                         
prot-a-1005 ukb-a-577              Weighted median     4 -0.002231  0.000963   
            ukb-a-577    Inverse variance weighted     4 -0.001807  0.000800   
prot-a-1045 ukb-a-92               Weighted median     4  0.001381  0.000629   
            ukb-a-92     Inverse variance weighted     4  0.001202  0.000521   
prot-a-1072 ukb-b-175              Weighted median     3 -0.001082  0.000526   
...                                            ...   ...       ...       ...   
prot-a-794  ukb-a-92     Inverse variance weighted     6 -0.001135  0.000343   
            ukb-b-11282            Weighted median     3 -0.000938  0.000476   
            ukb-b-11282  Inverse variance weighted     3 -0.000911  0.000432   
prot-a-857  ukb-b-15904            Weighted median     3 -0.001982  0.000843   
            ukb-b-15904  Inverse variance weighted     3 -0.001484  0.000732   

                             pval  
id.exposure id.outcome             
prot-a-1005 ukb-a-577    0.020535  
            ukb-a-577    0.023938  
prot-a-1045 ukb-a-92     0.028132  
            ukb-a-92     0.021127  
prot-a-1072 ukb-b-175    0.039629  
...                           ...  
prot-a-794  ukb-a-92     0.000944  
            ukb-b-11282  0.048565  
            ukb-b-11282  0.034799  
prot-a-857  ukb-b-15904  0.018759  
            ukb-b-15904  0.042675  

[98 rows x 7 columns]

In [18]:
filt_res.to_csv(os.path.join(results_dir, 'prot_vs_prolapse_filtered.csv'))


In [29]:
filt_res = res.query("nsnp > 2").set_index(["id.exposure", "id.outcome"])
all_same_sgn_fun = lambda ser: (ser > 0).all() or (ser < 0).all()
all_same_sgn = filt_res.groupby(["id.exposure", "id.outcome"])['b'].agg(all_same_sgn_fun)
filt_res = filt_res.loc[all_same_sgn].sort_values(['nsnp', "id.exposure", "id.outcome", 'method'], ascending=[False, True, True, True])
filt_res.to_csv(os.path.join(results_dir, 'prot_vs_prolapse_filtered.csv'))


In [27]:
filt_res.nsnp.value_counts()


3    2630
4     620
5     190
6      50
7       5
8       5
Name: nsnp, dtype: int64